In [2]:
import cobra
import pandas as pd

In [3]:
model = cobra.io.read_sbml_model("../results/reconstructions/gapfilled_pto.xml")

The purpose of this notebook is to use the gapfilled model for making growth predictions on single carbon sources. To do this, I will need to generate media conditions using the 192 carbon sources and the Biolog salts. Then, I need to run FBA and see if the model loaded is able to produce growth given the single carbon source. This will be a simple yes no to growth.

In [4]:
#define the base biolog media first
#first is the base composition, these are the salts that are necessary for growth, but alone do not contribute to growth
#/ minimally so
biolog_salts = pd.read_csv("../data/biolog_base_composition.csv")
print (biolog_salts)

         Name          ID
0         H2O  cpd00001_e
1          O2  cpd00007_e
2   Phosphate  cpd00009_e
3         CO2  cpd00011_e
4         NH3  cpd00013_e
5        Mn2+  cpd00030_e
6        Zn2+  cpd00034_e
7     Sulfate  cpd00048_e
8        Cu2+  cpd00058_e
9        Ca2+  cpd00063_e
10         H+  cpd00067_e
11        Cl-  cpd00099_e
12       Co2+  cpd00149_e
13         K+  cpd00205_e
14         Mg  cpd00254_e
15        Na+  cpd00971_e
16       Fe2+  cpd10515_e
17        fe3  cpd10516_e


In [6]:
biolog_medias = pd.read_csv("../data/biologplates.csv")
print (biolog_medias)

                                Name          ID
0                         m-Inositol  cpd00121_e
1           gamma-Amino Butyric Acid  cpd00281_e
2                        D-Galactose  cpd00108_e
3                        L-Glutamine  cpd00053_e
4                          L-Proline  cpd00129_e
5    D-Galactonic Acid-gamma-Lactone  cpd02143_e
6                         D-Mannitol  cpd00314_e
7                          D-Mannose  cpd00138_e
8                    alpha-D-Glucose  cpd00027_e
9                        L-Arabinose  cpd00224_e
10                           Sucrose  cpd00076_e
11               L-Pyroglutamic Acid  cpd01293_e
12                   L-Glutamic Acid  cpd00023_e
13                          D-Xylose  cpd00154_e
14                         L-Alanine  cpd00035_e
15                           Uridine  cpd00249_e
16                   D-Gluconic Acid  cpd00222_e
17                        Mucic Acid  cpd00652_e
18                      D-malic acid  cpd00386_e
19                  

In [7]:
model.medium

{'EXcpd00794_e': 1000.0,
 'EXcpd00023_e': 1000.0,
 'EXcpd00034_e': 1000.0,
 'EXcpd00041_e': 1000.0,
 'EXcpd00048_e': 1000.0,
 'EXcpd00053_e': 1000.0,
 'EXcpd00080_e': 1000.0,
 'EXcpd00107_e': 1000.0,
 'EXcpd00118_e': 1000.0,
 'EXcpd00119_e': 1000.0,
 'EXcpd00154_e': 1000.0,
 'EXcpd00156_e': 1000.0,
 'EXcpd00210_e': 1000.0,
 'EXcpd00264_e': 1000.0,
 'EXcpd00322_e': 1000.0,
 'EXcpd00637_e': 1000.0,
 'EXcpd00205_e': 1000.0,
 'EXcpd00971_e': 1000.0,
 'EXcpd00067_e': 1000.0,
 'EXcpd00060_e': 1000.0,
 'EXcpd00129_e': 1000.0,
 'EXcpd00027_e': 1000.0,
 'EXcpd00054_e': 1000.0,
 'EXcpd00009_e': 1000.0,
 'EXcpd00001_e': 1000.0,
 'EXcpd00013_e': 1000.0,
 'EXcpd00122_e': 1000.0,
 'EXcpd11596_e': 1000.0,
 'EXcpd01012_e': 1000.0,
 'EXcpd00058_e': 1000.0,
 'EXcpd11593_e': 1000.0,
 'EXcpd11592_e': 1000.0,
 'EXcpd11591_e': 1000.0,
 'EXcpd11590_e': 1000.0,
 'EXcpd11589_e': 1000.0,
 'EXcpd11588_e': 1000.0,
 'EXcpd01017_e': 1000.0,
 'EXcpd11587_e': 1000.0,
 'EXcpd11586_e': 1000.0,
 'EXcpd11585_e': 1000.0,
